<a href="https://colab.research.google.com/github/tesnimkh/Predicting-city-collaboration-with-business/blob/main/Predicting_city_collaboration_with_business.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S R
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

DATA_SOURCE_MAPPING = 'co2-emissions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F930654%2F1574269%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240619%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240619T174937Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D12d6220fea920daa2b48ecfa436b6292cb81ae926f8925da4ffd511a91d679ca5b0d6645c9229f7dbe5c3a153a10957b8ffff6c94a384cd9d1dc62c52378447c16c19363924533ae6c20e3645d94323e8f90d72b8eb069935a6f47ec820f68668f9faf70a40040aa26aabb6534706a5d21c3351da74e10531c80a51d95a5536a2107a16de3adc9e3156c185fe1b80d252d79ba949c7c0f812fe6352762fc9be91b8f2c25ad67e7b8504c10a6c7ee283287d1617415645fa777c92642e14f601af61f41537a5e8f0bdc6383896c76fc1584257077e93a4964f2f50871ed863ecc5fc2a513cf8b69ca8f6771cd737f7149dfe52c9f74924e458f983dd951a832aa'

KAGGLE_INPUT_PATH = '/kaggle/input'
KAGGLE_WORKING_PATH = '/kaggle/working'

system(paste0('sudo umount ', '/kaggle/input'))
system(paste0('sudo rmdir ', '/kaggle/input'))
system(paste0('sudo mkdir -p -- ', KAGGLE_INPUT_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_INPUT_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_INPUT_PATH,' ',file.path('..', 'input')),
  intern=TRUE)

system(paste0('sudo mkdir -p -- ', KAGGLE_WORKING_PATH), intern=TRUE)
system(paste0('sudo chmod 777 ', KAGGLE_WORKING_PATH), intern=TRUE)
system(
  paste0('sudo ln -sfn ', KAGGLE_WORKING_PATH, ' ', file.path('..', 'working')),
  intern=TRUE)

data_source_mappings = strsplit(DATA_SOURCE_MAPPING, ',')[[1]]
for (data_source_mapping in data_source_mappings) {
    path_and_url = strsplit(data_source_mapping, ':')
    directory = path_and_url[[1]][1]
    download_url = URLdecode(path_and_url[[1]][2])
    filename = sub("\\?.+", "", download_url)
    destination_path = file.path(KAGGLE_INPUT_PATH, directory)
    print(paste0('Downloading and uncompressing: ', directory))
    if (endsWith(filename, '.zip')){
      temp = tempfile(fileext = '.zip')
      download.file(download_url, temp)
      unzip(temp, overwrite = TRUE, exdir = destination_path)
      unlink(temp)
    }
    else{
      temp = tempfile(fileext = '.tar')
      download.file(download_url, temp)
      untar(temp, exdir = destination_path)
      unlink(temp)
    }
    print(paste0('Downloaded and uncompressed: ', directory))
}

print(paste0('Data source import complete'))


## Load packages

In [ ]:
install.packages("RTextTools")
library(dplyr)
library(ggplot2)
library(RTextTools)
library(h2o)
library(caret)


## Part 1: Data

In [ ]:
# Data from : https://ourworldindata.org/grapher/annual-co2-emissions-per-country?tab=chart
annual.co2.emissions.per.country <- read.csv("../input/co2-emissions/annual-co2-emissions-per-country.csv")

cities_responses_2020 <- read.csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Responses/2020_Full_Cities_Dataset.csv")

cities_disclosing_2020 <-read.csv("../input/cdp-unlocking-climate-solutions/Cities/Cities Disclosing/2020_Cities_Disclosing_to_CDP.csv")


In [ ]:
corporations_responses_2020 <-read.csv("../input/cdp-unlocking-climate-solutions/Corporations/Corporations Responses/Climate Change/2020_Full_Climate_Change_Dataset.csv")


corporations_disclosing_2020 <-read.csv("../input/cdp-unlocking-climate-solutions/Corporations/Corporations Disclosing/Climate Change/2020_Corporates_Disclosing_to_CDP_Climate_Change.csv")

## Part 2 : Research Question

We want to view the opportunities of cities and corporate corporations identified against climate change through the functions:

* printCityOrganization (country, org)
* printOpportunitiesCorporation (org)

and we want to build a model to predict the type of response that a city organization gives to the question:

Does your city collaborate in partnership with businesses in your city on sustainability projects?

with the following possible answers:

* Yes
* In progress
* Intending to undertake in the next 2 years
* Not intending to undertake
* Do not know

## Part 3 : Exploratory Data Analysis

In [ ]:
annual.co2.emissions.per.country %>%
  filter(Year==2018 & !Entity %in% c("World","Asia and Pacific (other)","EU-28","Middle East","Americas (other)","Europe (other)","International transport","Statistical differences","Africa","Asia (excl. China & India)","EU-28" ,"Europe","Europe (excl. EU-27)" ,"Europe (excl. EU-28)" ,"International transport" ,"North America (excl. USA)"  ,"North America", "Non KP Annex B"  ,"Non KP Annex B","Non-OECD","OECD" ,"KP Annex B","Asia","EU-27" ,"South America" )) %>%
  mutate(Entity=reorder(Entity,Annual.CO2.emissions))  %>%
  top_n(40) %>%
   ggplot(aes(Entity,Annual.CO2.emissions, fill=Entity))+
  geom_bar(stat="identity")+
  coord_flip()+
  guides(fill=FALSE)+
  geom_text(aes(label=round(Annual.CO2.emissions,2)), hjust=0, size=2)+
  ylab("CO2 milions tonnes") +
  xlab("Country") +
  ggtitle("CO2 emissions per Country in 2018")

In [ ]:
cities_disclosing_2020 %>%
  group_by(Country) %>%
  summarise(total=n()) %>%
  mutate(Country=reorder(Country,total))  %>%
  top_n(40) %>%
   ggplot(aes(Country,total, fill=Country))+
  geom_bar(stat="identity")+
  coord_flip()+
  guides(fill=FALSE)+
  geom_text(aes(label=total), hjust=0, size=2)+
  ylab("Total") +
  xlab("Country") +
  ggtitle("Cities organizations disclosing in 2020 by Country")

In [ ]:
corporations_disclosing_2020 %>%
  group_by(country) %>%
  summarise(total=n()) %>%
  mutate(country=reorder(country,total))  %>%
  ggplot(aes(country,total, fill=country))+
  geom_bar(stat="identity")+
  coord_flip()+
  guides(fill=FALSE)+
  geom_text(aes(label=total), hjust=0, size=3)+
  ylab("Total") +
  xlab("Country") +
  ggtitle("Corporations disclosing in 2020 per Country")

In [ ]:
printOpportunitiesCorporation <- function(org){
  df3<-corporations_disclosing_2020 %>%
        filter(organization==org) %>%
        select(country, primary_industry,primary_sector,primary_activity)

          cat("COUNTRY:",as.character(df3$country),"CORPORATION: ",org,"\n\n")

          cat(paste("PRIMARY INDUSTRY:",df3$primary_industry,", PRIMARY SECTOR:",df3$primary_sector,", PRIMARY ACTIVITY:",df3$primary_activity,"\n\n"))


  df2<-corporations_responses_2020 %>%
        filter(organization==org, question_number=="C2.4a") %>%
        select(question_unique_reference,response_value)

      cat(org,"OPPORTUNITIES :\n\n")
      cat("QUESTION: ",as.character(unique(df2$question_unique_reference)),"\n\n")
      print(paste("RESPONSE: ",df2$response_value))


}

In [ ]:
printPartnership <- function(df){

      if(nrow(df)!=0) {

          for (org in df$organization) {

              printOpportunitiesCorporation(org)

              cat("\n\n******************************************************\n\n")
          }

      }

}

In [ ]:
printCorporations <- function(c,opportunity){
  cat("\n\nCORPORATIONS BY OPPORTUNITIES:\n\n")
  if (opportunity %in% c("Development of clean energy technologies/businesses","Development of energy efficiency measures and technologies","Increased energy security")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry=="Power generation" , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Development of waste management sector")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_activity =="Waste management" , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Development of tourism industry and eco-tourism sector")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry =="Hospitality" , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Development of sustainable transport sector")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry =="Transportation services"  , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Development of sustainable construction/real estate sector")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_sector =="Construction"   , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)
  }
  else if (opportunity %in% c("Development of local sustainable food businesses","Increased food security","Extended agricultural seasons")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry =="Food, beverage & agriculture"   , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Increased opportunities for investment in infrastructure projects")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry =="Infrastructure" , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  else if (opportunity %in% c("Increased opportunities for trade (nationally or internationally)")){
    cat(toupper(opportunity),"\n\n")
    df2<-corporations_disclosing_2020 %>%
        filter(primary_industry =="Retail" , country==c) %>%
        select(organization,primary_industry,primary_sector,primary_activity)

  }
  printPartnership(df2)
}

In [ ]:
printCountryOpportunities <- function(country) {


  df_org<-cities_responses_2020 %>%
    filter(Question.Number=="6.0", Column.Name=="Opportunity", Country==country, ) %>%
    select(Organization)


  for (org in unique(df_org$Organization)) {
    df<-cities_responses_2020 %>%
       filter(Question.Number =="6.0", Column.Name=="Opportunity", Country==country, Organization==org) %>%
       select(Question.Name, Response.Answer)

    df_city<- cities_disclosing_2020 %>%
      filter(Country==country,Organization==org) %>%
      select(City)
    city <-df_city$City

    if (city=="") next

    cat("_____________________________________________________________\n\n")
    cat(paste("COUNTRY:",country,", CITY:",city,", ORGANIZATION:",org,"\n\n"))

    cat("QUESTION: ",unique(as.character(df$Question.Name)),"\n\n")
    cat("OPPORTUNITIES:\n\n")
    opportunities <-unique(as.character(df$Response.Answer))
    print(opportunities)

    cat("\n\n")
  }
}

In [ ]:
printCityOrganization <- function(country, org) {
      df<-cities_responses_2020 %>%
         filter(Question.Number =="6.0", Column.Name=="Opportunity", Country==country, Organization==org) %>%
         select(Question.Name, Response.Answer)

      df_city<- cities_disclosing_2020 %>%
        filter(Country==country,Organization==org) %>%
        select(City)
      city <-df_city$City

      if (city!="") {

          cat("_____________________________________________________________\n\n")
          cat(paste("COUNTRY:",country,", CITY:",city,", ORGANIZATION:",org,"\n\n"))

          cat("QUESTION: ",unique(as.character(df$Question.Name)),"\n\n")
          cat("OPPORTUNITIES:\n\n")
          opportunities <-unique(as.character(df$Response.Answer))
          print(opportunities)

          cat("\n\n")
      }
    }

In [ ]:
#printOpportunitiesCorporation("TransAlta Corporation")

In [ ]:
#printCountryOpportunities("China")

In [ ]:
printCountryOpportunities("China, Hong Kong Special Administrative Region")

In [ ]:
#printCorporations("United States of America","Development of clean energy technologies/businesses")

In [ ]:
printCityOrganization("China","Chengdu Municipal Government")


## Part 4: Modeling

A dataset is built with the cities' answers to the questions:

6.0 Please indicate the opportunities your city has identified as a result of addressing climate change and describe how the city is positioning itself to take advantage of these opportunities.

Select from:
* Development of clean energy technologies / businesses Text field
* Development of energy efficiency measures and technologies
* Development of waste management sector
* Development of water management sector
* Development of circular economy models and businesses
* Development of tourism industry and eco-tourism sector
* Development of sustainable transport sector
* Development of sustainable construction / real estate sector
* Development of local sustainable food businesses
* Increased opportunities for investment in infrastructure projects
* Increased opportunities for trade (nationally or internationally)
* Additional funding opportunities
* Increase opportunities for partnerships
* Improved flood risk mitigation
* Increased water security
* Increased energy security
* Development of climate change resiliency projects
* Increased food security
* Extended agricultural seasons
* Reduced risk to natural capital
* Reduced risk to human health
* Creation / development of carbon markets
* Carbon tax revenue
* Development of resource conservation and management
* Improved efficiency of municipal operations
* Other, please specify
* No opportunities identified

6.2 Does your city collaborate in partnership with businesses in your city on sustainability projects?

* Yes
* In progress
* Intending to undertake in the next 2 years
* Not intending to undertake
* Do not know


The following numerical variables are added to the dataset: current population of the city and size of the territory in square km.

The answer to question 6.2 is the target to predict.

In the sample, the total per response is given by the following plot:


In [ ]:
df<-cities_responses_2020 %>%
       filter(Question.Number =="6.2") %>%
       select(Country,Organization,Response.Answer)

df$Response.Answer <- as.factor(as.character(df$Response.Answer))

df %>%
  group_by(Response.Answer) %>%
  summarise(total=n()) %>%
  mutate(Response.Answer=reorder(Response.Answer,total))  %>%
  ggplot(aes(Response.Answer,total, fill=Response.Answer))+
  geom_bar(stat="identity")+
  coord_flip()+
  guides(fill=FALSE)+
  geom_text(aes(label=total), hjust=0, size=2)+
  ylab("Total") +
  xlab("Answer") +
  ggtitle("Does your city collaborate in partnership with businesses",subtitle =  " in your city on sustainability projects?")

### H2o Model

In [ ]:
df_cities <- cities_disclosing_2020
df<-cities_responses_2020 %>%
       filter(Question.Number =="0.5", Column.Name=="Current population") %>%
       select(Country,Organization,Response.Answer)
df_cities <- merge(df_cities,df, by = c("Country","Organization"))
df_cities$Response.Answer <- as.numeric(as.character(df_cities$Response.Answer))
colnames(df_cities)[14] <- "Current population"


df<-cities_responses_2020 %>%
       filter(Question.Number =="0.6", Column.Name=="Land area of the city boundary as defined in question 0.1 (in square km)") %>%
       select(Country, Organization,Response.Answer)
df_cities <- merge(df_cities,df, by = c("Country","Organization"))
df_cities$Response.Answer <- as.numeric(as.character(df_cities$Response.Answer))
colnames(df_cities)[15] <- "Land area square Km"


df<-cities_responses_2020 %>%
       filter(Question.Number =="6.2") %>%
       select(Country,Organization,Response.Answer)
df_cities <- merge(df_cities,df, by = c("Country","Organization"))
colnames(df_cities)[16] <- "target"
df_cities$target <- as.factor(as.character(df_cities$target))


df<-cities_responses_2020 %>%
       filter(Question.Number =="6.0", Column.Name=="Opportunity") %>%
       select(Country,Organization,Response.Answer)
df_cities <- merge(df_cities,df, by = c("Country","Organization"))
colnames(df_cities)[17] <- "opportunity"
df_cities$opportunity <- as.character(df_cities$opportunity)
df_cities$Country <- as.character(df_cities$Country)



The non-existent responses from the target are deleted:

In [ ]:
df_cities <-df_cities[-which(df_cities$target==""),]
df_cities$target <- as.factor(as.character(df_cities$target))

The h2o model is created:

In [ ]:
h2o.init()

We build a training set made from 70% of the observations on which the model is trained and a testing set made from the remaining 30% of observations on which the model is tested:

In [ ]:
train <- createDataPartition(df_cities$target,p=0.7,list = FALSE)
training_set <- df_cities[train,]
testing_set <- df_cities[-train,]

In [ ]:
df_hf <- as.h2o(training_set)
y <- "target"
x <- names(training_set)[c(1,14,15,17)]

aml <- h2o.automl(x = x, y = y,
                  training_frame = df_hf,
                  max_runtime_secs = 30)

In [ ]:
lb <- aml@leaderboard
lb

The leader model found is tested on the testing set:

In [ ]:
test <- as.h2o(testing_set)
model <- aml@leader
p1 = h2o.predict(model, newdata=test)

The accuracy is calculated on the testing set:

In [ ]:
df2 <- as.data.frame(p1$predict)

paste("Accuracy=", mean(df2$predict==testing_set$target))

The confusion matrix is constructed :

In [ ]:
confusionMatrix(df2$predict,testing_set$target)

### Sentiment Analysys Model:

In [ ]:
df1<-cities_responses_2020 %>%
         filter(Question.Number =="6.0", Column.Name=="Opportunity", ) %>%
         select(Country,Organization, Response.Answer)

df2<-cities_responses_2020 %>%
         filter(Question.Number =="6.2") %>%
         select(Country,Organization, Response.Answer)


df1 <- merge(df1,df2 , by = c("Country","Organization"))
colnames(df1)[3:4] <- c("opportunity","target")
df1$opportunity <- as.factor(as.character(df1$opportunity))
df1$target <- as.factor(as.character(df1$target))

In [ ]:
head(df1)

A sentyment analysis model is built through a Boosting algorithm to predict the answer to the question:

6.2 Does your city collaborate in partnership with businesses in your city on sustainability projects?

In particular, the training set is 1500 observations, while the testing set is 1077 observations.

The data on which the sentyment analysis model is built are: Country and opportunity of the dataset df1.

In [ ]:
data <- df1[sample(1:2577,size=2577,replace=FALSE),]
matrix <- create_matrix(cbind(data["Country"],data["opportunity"]), language="english",
removeNumbers=TRUE, stemWords=FALSE)
container <- create_container(matrix,as.numeric(data$target),trainSize=1:1500, testSize=1501:2577,
virgin=FALSE)

In [ ]:
models = train_models(container, algorithms=c("BOOSTING"))

results = classify_models(container, models)

In [ ]:
analytics = create_analytics(container, results)
summary(analytics)

As you can see, the Accuracy of the forecast is 82.91%

In [ ]:
paste("Accuracy=",mean(analytics@document_summary$MANUAL_CODE==
analytics@document_summary$LOGITBOOST_LABEL))

## Part 6: Prediction

In [ ]:
df_test <- data_frame(Country="Canada",`Current population`=74003,`Land area square Km`=329, opportunity="Development of sustainable construction/real estate sector")

df_test <- as.h2o(df_test)
p1 = h2o.predict(model, newdata=df_test)

p2<-as.character(p1$predict)

head(df_test)
paste("Does your city collaborate in partnership with businesses in your city on sustainability projects?", p2$predict[1,1])


df_test <- data_frame(Country="Turkey",`Current population`=482713,`Land area square Km`=25.2, opportunity="Reduced risk to human health")

df_test <- as.h2o(df_test)
p1 = h2o.predict(model, newdata=df_test)

p2<-as.character(p1$predict)

head(df_test)
paste("Does your city collaborate in partnership with businesses in your city on sustainability projects?", p2$predict[1,1])